# REGRESIÓN MULTILINEAR

In [0]:
# Definición de contexto
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [0]:
# Importamos librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## CARGA DEL DATASET

In [0]:
#Loading data
loaded_data = spark.read.csv('/FileStore/tables/weatherHistory.csv',inferSchema=True, header=True)
#To check the columns of dataframe
loaded_data.columns

Out[3]: ['Formatted Date',
 'Summary',
 'Precip Type',
 'Temperature (C)',
 'Apparent Temperature (C)',
 'Humidity',
 'Wind Speed (km/h)',
 'Wind Bearing (degrees)',
 'Visibility (km)',
 'Loud Cover',
 'Pressure (millibars)',
 'Daily Summary']

## PREPARACIÓN DE LOS DATOS

En este apartado, se realiza la adecuación de los datos que acabamos de cargar y se realizan diversos procesos que, seguidamente, se pasan a comentar paso a paso revisando qué hace cada parte del bloque de código:

1. **VectorAssembler**:
   - `VectorAssembler` es una transformación que combina una lista dada de columnas en una única columna de vectores. Esto es útil para combinar características en un formato que los algoritmos de aprendizaje automático pueden consumir.
   - `inputCols`: especifica las columnas del dataframe que se quieren incluir como características.
   - `outputCol`: especifica el nombre de la nueva columna que contendrá el vector de características.
   - En este caso, `get_assembler` toma las columnas especificadas como temperatura, humedad, velocidad del viento, etc., y las combina en una nueva columna llamada `get_feature`.

2. **Transformación y visualización**:
   - `get_assembler.transform(loaded_data)` aplica la transformación `VectorAssembler` al dataframe `loaded_data`.
   - `op_assembler.show()` muestra las primeras filas del dataframe transformado para verificar que la nueva columna `get_feature` se ha añadido correctamente.

3. **StringIndexer**:
   - `StringIndexer` convierte una columna de etiquetas de cadena en etiquetas de índice numérico. Esto es necesario porque muchos algoritmos de ML en Spark esperan que las etiquetas sean numéricas.
   - `inputCol`: la columna que contiene las etiquetas de cadena.
   - `outputCol`: la columna resultante con índices numéricos.
   - `get_indexer = StringIndexer(inputCol='Summary', outputCol='summary_index')` crea un `StringIndexer` que transformará la columna `Summary` en una columna de índices numéricos llamada `summary_index`.

4. **Aplicación de StringIndexer y selección de columnas**:
   - `finalized_data = get_indexer.fit(op_assembler).transform(op_assembler)` ajusta el `StringIndexer` al dataframe `op_assembler` y luego lo transforma, añadiendo la columna `summary_index`.
   - `finalized_data = finalized_data.select("get_feature", "summary_index")` selecciona solo las columnas de características vectorizadas y las etiquetas indexadas, dejando de lado otras columnas del dataframe original.

5. **División de datos**:
   - `training_data, testing_data = finalized_data.randomSplit([0.7,0.3])` divide el dataframe en dos subconjuntos: uno para entrenamiento (70% de los datos) y otro para prueba (30% de los datos). Esto se hace comúnmente para poder entrenar modelos en un conjunto de datos y evaluarlos en otro independiente para verificar su desempeño.


En resumen, el siguiente bloque de código prepara un conjunto de datos para un proceso de modelado supervisado, organizando y transformando las características y etiquetas de manera adecuada para su uso en modelos de aprendizaje automático en Spark.

In [0]:
#Conversión a un vector 
get_assembler = VectorAssembler(inputCols=['Temperature (C)',
 'Apparent Temperature (C)',
 'Humidity',
 'Wind Speed (km/h)',
 'Wind Bearing (degrees)',
 'Visibility (km)',
 'Loud Cover',
 'Pressure (millibars)'],outputCol='get_feature')
op_assembler = get_assembler.transform(loaded_data)
op_assembler.show()
get_indexer = StringIndexer(inputCol='Summary', outputCol='summary_index')
finalized_data = get_indexer.fit(op_assembler).transform(op_assembler)
finalized_data = finalized_data.select("get_feature", "summary_index")
training_data, testing_data = finalized_data.randomSplit([0.7,0.3])

+-------------------+-------------+-----------+------------------+------------------------+--------+------------------+----------------------+------------------+----------+--------------------+--------------------+--------------------+
|     Formatted Date|      Summary|Precip Type|   Temperature (C)|Apparent Temperature (C)|Humidity| Wind Speed (km/h)|Wind Bearing (degrees)|   Visibility (km)|Loud Cover|Pressure (millibars)|       Daily Summary|         get_feature|
+-------------------+-------------+-----------+------------------+------------------------+--------+------------------+----------------------+------------------+----------+--------------------+--------------------+--------------------+
|2006-03-31 22:00:00|Partly Cloudy|       rain| 9.472222222222221|      7.3888888888888875|    0.89|           14.1197|                 251.0|15.826300000000002|       0.0|             1015.13|Partly cloudy thr...|[9.47222222222222...|
|2006-03-31 23:00:00|Partly Cloudy|       rain| 9.355555

## CREACIÓN, ENTRENAMIENTO Y EVALUACIÓN DEL MODELO DE REGRESIÓN

El siguiente bloque de código implementa y evalúa un modelo de regresión lineal. A continuación se comenta cada uno de sus apartados:

1. **Creación del Modelo de Regresión Lineal**:
   - `LinearRegression(featuresCol="get_feature", labelCol="summary_index")`:  crea una instancia del modelo de regresión lineal. El argumento `featuresCol` especifica el nombre de la columna que contiene las características del modelo (en este caso, la columna de vectores creada por `VectorAssembler`), y `labelCol` especifica la columna que contiene la variable objetivo o etiqueta (los índices numéricos de la columna `Summary` creados por `StringIndexer`).

2. **Entrenamiento del Modelo**:
   - `lr_model = lr.fit(training_data)`: Esta línea entrena el modelo de regresión lineal utilizando los datos de entrenamiento (`training_data`). El modelo resultante (`lr_model`) incluirá los coeficientes (pesos) y el término de interceptación aprendidos a partir de los datos de entrenamiento.

3. **Evaluación del Modelo**:
   - `test_results = lr_model.evaluate(testing_data)`: Después de entrenar el modelo, este se evalúa utilizando el conjunto de datos de prueba (`testing_data`). El método `evaluate` aplica el modelo entrenado a los datos de prueba para predecir las etiquetas y calcular métricas de evaluación.
   - `test_results.residuals.show()`: Los residuos son las diferencias entre los valores observados de la variable dependiente en el conjunto de datos de prueba y los valores predichos por el modelo. Esta línea muestra los residuos de cada punto en el conjunto de datos de prueba. La visualización de residuos puede ayudar a diagnosticar problemas con el modelo, como si los errores son consistentemente altos en ciertas áreas, lo que puede indicar un ajuste insuficiente o la presencia de valores atípicos.

En resumen, este bloque de código entrena un modelo de regresión lineal sobre un conjunto de características y etiquetas, evalúa su desempeño en un conjunto de datos de prueba, y luego muestra los residuos para analizar la precisión de las predicciones del modelo. La visualización de los residuos es una parte crucial del análisis de regresión, ya que proporciona información sobre la adecuación del modelo y la naturaleza de los errores que comete.

#### NOTA

El término de **interceptación** en un modelo de regresión lineal, también conocido como el término independiente o el coeficiente de intercepción, es un componente fundamental del modelo. Aquí te explico qué es y qué indica:

1. **Qué es el término de interceptación**:
   - En la ecuación de un modelo de regresión lineal, \( y = b_0 + b_1x_1 + b_2x_2 + \cdots + b_nx_n \), donde \( y \) es la variable dependiente, \( x_1, x_2, \ldots, x_n \) son las variables independientes, y \( b_1, b_2, \ldots, b_n \) son los coeficientes de esas variables, el término \( b_0 \) es el intercepto.
   - Este coeficiente \( b_0 \) representa el valor esperado de \( y \) cuando todas las variables independientes (\( x \)) son iguales a cero. 

2. **Qué indica el término de interceptación**:
   - **Punto de partida**: El intercepto muestra dónde la línea de regresión intercepta el eje \( y \) en un gráfico de dispersión cuando todas las variables independientes son cero. Es el valor base de la variable dependiente antes de que se considere cualquier influencia de las variables independientes.
   - **Valor esperado base**: Si todas las variables predictoras son cero, el intercepto es el valor promedio esperado de la variable respuesta. Por ejemplo, en un contexto de negocio, si el modelo predice ventas basadas en gastos de publicidad y precio del producto, el intercepto representaría las ventas esperadas cuando los gastos de publicidad y el precio son cero (suponiendo que sea un escenario posible).
   - **Bias o sesgo del modelo**: Sin el término de interceptación, el modelo forzaría la línea de regresión a pasar por el origen (0,0), lo que puede no ser adecuado o realista para muchos tipos de datos y podría resultar en un modelo sesgado.

En resumen, el término de **interceptación** es esencial para ajustar correctamente un modelo de regresión a los datos. Permite que el modelo se alinee adecuadamente con los datos, proporcionando un punto de partida que refleja la influencia de tener todas las variables independientes en cero. Además, ayuda a interpretar el modelo, ofreciendo una línea de base desde la cual se pueden medir los efectos de las variables predictoras.

In [0]:
#CREACIÓN DEL MODELO
lr = LinearRegression(featuresCol="get_feature", labelCol="summary_index",)
#ENTRENAMIENTO DEL MODELO
lr_model = lr.fit(training_data)
#EVALUACIÓN DEL MODELO
test_results = lr_model.evaluate(testing_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 0.5896005541915543|
| 0.6991642906989717|
| 0.9211068106960454|
| 0.5035256849566645|
|   1.06807477198541|
| 1.2558119396123668|
|0.46118098981015443|
| 1.2676181533810533|
|   1.17914390204034|
| 1.2781356771689318|
|-1.7493674029641388|
| 1.1206908258973693|
|0.23472835642952283|
|0.44601858852399867|
| 1.2891959862362974|
|  1.249596077313527|
| 0.3504283295639927|
|-1.2733631644766095|
| 1.2661017947341178|
|-1.0689118539883173|
+-------------------+
only showing top 20 rows



### EVALUACIÓN SOBRE CONJUNTO DE TEST
El siguiente bloque de código lleva a cabo varias operaciones relacionadas con la evaluación de un modelo de regresión lineal en Spark y el análisis preliminar de los datos de entrenamiento. A continuación, se detalla cada paso y su función:

1. **Evaluación del Modelo con Datos de Prueba**:
   - `get_prediction = lr_model.transform(testing_data)`: Esta línea utiliza el modelo de regresión lineal `lr_model` entrenado previamente para predecir la variable dependiente (denominada `summary_index` en este contexto) utilizando el conjunto de datos de prueba (`testing_data`). El método `transform` aplica el modelo a los datos de prueba y añade una columna de predicciones al DataFrame.
   - `get_prediction.show()`: Muestra las primeras filas del DataFrame de resultados, incluyendo las predicciones hechas por el modelo. Esto permite ver rápidamente cómo el modelo está realizando predicciones con respecto a los datos reales y cualquier otra columna que esté en el DataFrame.

2. **Análisis Descriptivo de los Datos de Entrenamiento**:
   - `training_data.describe().show()`: Este comando genera estadísticas descriptivas para todas las columnas numéricas en el DataFrame de datos de entrenamiento (`training_data`). Esto incluye contar, media, desviación estándar, mínimo y máximo. Es útil para obtener una visión general de la distribución de los datos, detectar valores atípicos, y entender las escalas de las variables.

3. **Preparación de Datos para Análisis o Visualización en Python**:
   - `train = training_data.select("get_feature","summary_index").toPandas()`: Esta línea selecciona solo las columnas "get_feature" y "summary_index" del DataFrame de Spark y las convierte a un DataFrame de pandas. Esto es comúnmente realizado para facilitar análisis más detallados o visualizaciones utilizando bibliotecas de Python, que a menudo tienen más opciones y flexibilidad que las disponibles directamente en Spark.
   - `train_get_feature = train['get_feature']`: Extrae la columna "get_feature" del DataFrame de pandas `train` y la guarda en la variable `train_get_feature`. "get_feature" contiene vectores de características utilizadas en el modelo.
   - `train_get_feature = list(train_get_feature)`: Convierte la serie pandas que contiene los vectores de características a una lista de Python. Esto es útil para iteraciones o transformaciones posteriores.
   - `train_get_salary = train['summary_index']`: Extrae la columna "summary_index" del DataFrame de pandas y la guarda en la variable `train_get_salary`. Esta columna contiene los valores de la variable dependiente para los datos de entrenamiento.

Por tanto, este código evalúa el rendimiento del modelo de regresión lineal en el conjunto de datos de prueba, proporciona estadísticas descriptivas sobre los datos de entrenamiento y prepara los datos para análisis adicionales o visualización.

In [0]:
#Testing dataset on lr_model
get_prediction = lr_model.transform(testing_data)
get_prediction.show()
#Get_training_insights
training_data.describe().show()
train = training_data.select("get_feature","summary_index").toPandas()
train_get_feature = train['get_feature']
train_get_feature = list(train_get_feature)
train_get_salary = train['summary_index']

+--------------------+-------------+------------------+
|         get_feature|summary_index|        prediction|
+--------------------+-------------+------------------+
|(8,[0,1,2,5],[6.1...|          4.0|3.4103994458084457|
|(8,[0,1,2,5],[8.8...|          3.0|2.3008357093010283|
|(8,[0,1,2,5],[11....|          3.0|2.0788931893039546|
|(8,[2,3,4,5],[0.8...|          3.0|2.4964743150433355|
|[-14.088888888888...|          3.0|  1.93192522801459|
|[-14.033333333333...|          3.0|1.7441880603876332|
|[-14.022222222222...|          3.0|2.5388190101898456|
|[-13.0,-13.0,0.87...|          3.0|1.7323818466189467|
|[-12.850000000000...|          3.0|  1.82085609795966|
|[-12.805555555555...|          3.0|1.7218643228310682|
|[-12.711111111111...|          0.0|1.7493674029641388|
|[-12.688888888888...|          3.0|1.8793091741026307|
|[-12.577777777777...|          2.0|1.7652716435704772|
|[-12.55,-21.75,0....|          2.0|1.5539814114760013|
|[-12.422222222222...|          3.0|1.7108040137

/databricks/spark/python/pyspark/sql/pandas/conversion.py:122: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field get_feature. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT()
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


### PREDICCIÓN

El código siguiente realiza una serie de operaciones para evaluar el desempeño de un modelo de regresión lineal en el conjunto de datos de entrenamiento y preparar las predicciones para análisis o visualización adicional. Estos son los pasos que se llevan a cabo:

1. **Evaluación del Modelo en Datos de Entrenamiento**:
   - `get_training_prediction = lr_model.transform(training_data)`: Utiliza el modelo de regresión lineal (`lr_model`) entrenado anteriormente para hacer predicciones sobre el mismo conjunto de datos con el que fue entrenado (`training_data`). Esto es útil para verificar cómo el modelo se comporta con los datos que ya conoce.
   - El resultado es un DataFrame de Spark que incluye todas las columnas originales de `training_data` más una columna adicional llamada `prediction`, que contiene las predicciones del modelo para cada observación en el conjunto de datos de entrenamiento.

2. **Conversión de DataFrame de Spark a DataFrame de Pandas**:
   - `train_pred = get_training_prediction.select("prediction").toPandas()`: Selecciona solo la columna `prediction` del DataFrame de Spark y la convierte en un DataFrame de Pandas. Esta operación facilita el manejo y la visualización de los datos en Python al utilizar Pandas para el análisis de datos.

3. **Extracción y Conversión de Datos para Análisis o Visualización**:
   - `prediction_train = train_pred['prediction']`: Extrae la columna de predicciones del DataFrame de Pandas `train_pred` y la guarda en la variable `prediction_train`.
   - `prediction_list = list(prediction_train)`: Convierte la serie de Pandas que contiene las predicciones en una lista de Python. Esto es útil para operaciones que requieren estructuras de datos nativas de Python, como iterar sobre las predicciones, realizar cálculos adicionales o integrar con otras bibliotecas de Python para la visualización de datos.

4. **Impresión de la Lista de Predicciones**:
   - `print(prediction_list)`: Imprime la lista de predicciones, proporcionando una salida inmediata y directa de las predicciones generadas por el modelo para el conjunto de entrenamiento, que ayuda en la interpretación rápida de cómo el modelo está estimando los valores basados en las características aprendidas.

En definitiva, el código es especialmente útil para diagnosticar y entender cómo el modelo de regresión lineal está funcionando con los datos utilizados para su entrenamiento, permitiendo identificar si hay sobreajuste, subajuste, o cualquier patrón peculiar en las predicciones.

In [0]:
#Predicción
get_training_prediction = lr_model.transform(training_data)
#Conversión a dataframe de Panda para visualizar datos
train_pred = get_training_prediction.select("prediction").toPandas()
prediction_train = train_pred['prediction']
prediction_list = list(prediction_train)
print(prediction_list)

[3.4121314801439038, 3.268153015266007, 3.182760181903813, 3.1550636910890164, 2.0118806154047943, 2.695862107196835, 2.6920053750277124, 2.593695488345456, 3.828006237397143, 1.9824361280331169, 1.8566668687312657, 1.7906449551073877, 1.7609285711324234, 2.318388517809191, 1.161957273553583, 1.0357780479519392, 1.0702056632478676, 1.731205121385658, 1.6724354360908924, 1.965804597546059, 0.970527599132438, 2.0055242882591426, 1.7208873322909617, 1.9126852491171946, 2.282079220265483, 1.673223726304738, 2.496530636586659, 1.7128491279576736, 1.849385028895292, 1.0305654986019208, 1.6676539399464805, 1.5623678877351677, 1.6621649433010575, 1.2216394255526368, 1.1033099568214229, 1.5555539555401428, 1.127594246852925, 1.8179028322190818, 1.7518593782115466, 1.7483809962528223, 1.6553026024381254, 2.4690364884355116, 1.1075089076546396, 1.6774893467393661, 0.9716802674333787, 1.5939566864711936, 1.5950287985437475, 1.7699577442596688, 2.1413938068340377, 1.632071176778894, 1.5423602262145

### TEST

El siguiente código prepara los datos para una evaluación detallada del modelo, facilitando la visualización de las características y etiquetas reales del conjunto de prueba. Esto es útil para hacer comparaciones directas entre los valores predichos y los reales, analizar cómo el modelo está interpretando las características, y validar la efectividad del modelo en datos que no fueron utilizados durante el entrenamiento. Estas tareas son esenciales para entender y mejorar el rendimiento del modelo de regresión lineal.

1. **Selección y Conversión de Datos**:
   - `x = testing_data.select("get_feature", "summary_index").toPandas()`: Esta línea selecciona dos columnas específicas, `get_feature` y `summary_index`, del DataFrame de Spark `testing_data`, que contiene el conjunto de prueba. Luego, convierte este subconjunto de datos a un DataFrame de Pandas. Esta operación facilita la manipulación y visualización de los datos ya que Pandas ofrece herramientas de análisis más flexibles y detalladas que Spark.

2. **Extracción de Características y Etiquetas**:
   - `x_get = x['get_feature']`: Extrae la columna `get_feature` del DataFrame de Pandas `x` y la guarda en la variable `x_get`. Esta columna contiene los vectores de características utilizados por el modelo para hacer predicciones. Cada elemento de `get_feature` es un vector que representa características numéricas.
   - `y_get = x['summary_index']`: Extrae la columna `summary_index` del DataFrame de Pandas `x` y la guarda en la variable `y_get`. `summary_index` contiene las etiquetas o valores reales que el modelo intenta predecir, los cuales han sido transformados previamente a índices numéricos mediante `StringIndexer`.



In [0]:
#TEST
x = testing_data.select("get_feature","summary_index").toPandas()
x_get = x['get_feature']
y_get = x['summary_index']

/databricks/spark/python/pyspark/sql/pandas/conversion.py:122: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field get_feature. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Unsupported type in conversion to Arrow: VectorUDT()
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


In [0]:
#Obtención del resumen del modelo
print("Summary of model is here:")
lr_model.summary

Summary of model is here:
Out[9]: <pyspark.ml.regression.LinearRegressionTrainingSummary at 0x7fe290906160>

### COEFICIENTES E INTERCEPTO

Estos dos elementos de información permiten entender cómo el modelo de regresión lineal está haciendo sus predicciones. Por una parte, los **coeficientes** indican la influencia relativa de cada característica en la predicción, y el **intercepto** proporciona un punto de referencia básico para esas predicciones.   
En conjunto, coeficientes e intercepto forman la ecuación completa del modelo de regresión lineal que puede ser usada para hacer predicciones sobre nuevos datos.

In [0]:
#Obtención de los coeficientes y del intercepto
print("Coeficientes: " + str(lr_model.coefficients))
print("Intercepto: " + str(lr_model.intercept))

Coeficientes: [-0.07214915879122738,0.051924551009013525,1.1288896307584562,0.05126322660699349,-0.00030899343612469216,-0.11049797277716629,0.0,-0.0009546366755254025]
Intercepto: 2.503490113205684


### MÉTRICAS Y EVALUACIÓN

In [0]:
#Métricas de evaluación
eval = RegressionEvaluator(labelCol="summary_index", predictionCol="prediction", metricName="rmse")

In [0]:
# Root Mean Square Error
rmse = eval.evaluate(get_prediction)
print("RMSE: %.3f" % rmse)
# Mean Square Error
mse = eval.evaluate(get_prediction, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)
# Mean Absolute Error
mae = eval.evaluate(get_prediction, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)
# r2 - coefficient of determination
r2 = eval.evaluate(get_prediction, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 1.487
MSE: 2.210
MAE: 1.016
r2: 0.190


In [0]:

#Obtención de métricas directamente del 'model.summary'
print("Métricas extraídas del resumen del modelo:")
print("R2: ", lr_model.summary.r2)
print("Root Mean Squared Error: ", lr_model.summary.rootMeanSquaredError)
print("Mean Squared Error: ", lr_model.summary.meanSquaredError)
print("Mean Absolute Error: ", lr_model.summary.meanAbsoluteError)
print("Explained Variance: ", lr_model.summary.explainedVariance) # Sólo es posible extraerla desde el 'model.summary'


Métricas extraídas del resumen del modelo:
R2:  0.19511452471050805
Root Mean Squared Error:  1.4558514195662307
Mean Squared Error:  2.1195033558530088
Mean Absolute Error:  1.0026779371016865
Explained Variance:  0.513794699473004
